# Chapter1.3: Implementing a 5-Minute Agent

### Import packages

In [2]:
!pip install requests tavily-python openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.1/997.1 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [openai]━━━━ 3/4 [openai]


### (1) Prompt Engineering

In [3]:
SYS_PROMPT = '''
You are a smart traveling assistant. Your task is to analyse the requests from the user and solve them using a set of tools step by step.

# You have the access to the following tools:
- `get_weather(city: str) -> str`: Get the current weather information for a given city.
- `get_attractions(city: str, weather: str) -> str`: Get a list of popular tourist attractions in the specified city based on the current weather conditions.

# Task instructions:
Your answer should always follow the steps below. First, you need to describe your reasoning process. Second, state what exactly you want to do. Only response on pair of 'Thought - Action' pair per response.
Thought: [Your reasoning and next step plan here]
Action: The action you want to take, should be one of:
- `function_name(arg_name = "arg_value")`: Call an allowed function with the specified argument.
- `Finish[Final Answer]`: If you have completed all the necessary steps and have the final answer to provide to the user.
- When you have collected sufficient information to provide a comprehensive response to the user's request, you must answer Finish[Final Answer] after Action: field to outout the final answer.
'''

### (2) Tool1: Get Weather Information

In [4]:
import requests

def get_weather(city: str) -> str:
    '''
    Get the current weather information for a given city.
    '''

    url = f'http://wttr.in/{city}?format=j1'

    try:
        # Make a GET request to the weather API
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # Extract relevant weather information to form an observation
        current_condition = data['current_condition'][0]
        weather_desc = current_condition['weatherDesc'][0]['value']
        temp_c = current_condition['temp_C']

        # Return the formatted weather information
        return f'The current weather in {city} is {weather_desc} with a temperature of {temp_c}°C.'

    # Handle potential errors
    except requests.exceptions.RequestException as e:
        return f'Error fetching weather data: {e}'
    except (KeyError, IndexError) as e:
        return f'Error parsing weather data: {e}'

### (3) Tool 2: Search and Recommend Attractions

In [5]:
import os
from tavily import TavilyClient

def get_attractions(city: str, weather: str) -> str:
    '''
    Get a list of popular tourist attractions in the specified city based on the current weather conditions.
    '''

    api_key = os.getenv('TAVILY_API_KEY')
    if not api_key:
        return 'Tavily API key is not set in environment variables.'
    
    client = TavilyClient(api_key=api_key)

    query = f'List popular tourist attractions in {city} considering the current weather: {weather}.'

    try:

        response = client.search(query=query, search_depth='basic', include_answers=True)

        if response.get('answers'):
            return response['answers']
        
        formatted_results = [f"- {item['title']}: {item['content']}" for item in response.get('results', [])]

        if not formatted_results:
            return f'No attractions found for {city}.'
        
        return f"Based on the current weather, here are some popular tourist attractions in {city}:\n" + "\n".join(formatted_results)
    
    except Exception as e:
        return f'Error fetching attractions data: {e}'

In [6]:
available_tools = {
    'get_weather': get_weather,
    'get_attractions': get_attractions
}

### (5) LLM APIs

In [7]:
from openai import OpenAI

class OpenAICompatibleClient:
    """
    API Client for OpenAI-compatible LLM services.
    """
    def __init__(self, model: str, api_key: str, base_url: str):
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)

    def generate(self, prompt: str, system_prompt: str) -> str:
        """Call LLM API to generate a response."""
        try:
            messages = [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': prompt}
            ]
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                stream=False
            )
            answer = response.choices[0].message.content
            return answer
        except Exception as e:
            print(f"Error occurred while calling LLM API: {e}")
            return "Error: An error occurred while calling the language model service."

### (6) Main Loop

In [8]:
import re

# init environment variables
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME', 'gpt-4o')
OPENAI_API_BASE_URL = os.getenv('OPENAI_API_BASE_URL', 'https://api.openai.com/v1')

llm = OpenAICompatibleClient(
    model = OPENAI_MODEL_NAME,
    api_key = OPENAI_API_KEY,
    base_url = OPENAI_API_BASE_URL
)

user_prompt = "I am planning to visit Paris. What's the weather like there and what attractions should I visit?"
prompt_history = [f"User Request: {user_prompt}"]

print("Starting the agent...\n")
print(f"User: {user_prompt}\n")

# Agent loop

for i in range(5):
    print(f"--- Step {i+1} ---")

    full_prompt = "\n".join(prompt_history)
    llm_output = llm.generate(prompt=full_prompt, system_prompt=SYS_PROMPT)
    match = re.search(r'(Thought:.*?Action:.*?)(?=\n\s*(?:Thought:|Action:|Observation:)|\Z)', llm_output, re.DOTALL)
    if match:
        truncated = match.group(1).strip()
        if truncated != llm_output.strip():
            llm_output = truncated
            print("Truncated extra Thought-Action pairs")
    print(f"Agent Response:\n{llm_output}\n")
    prompt_history.append(llm_output)

    # parse and execute action
    action_match = re.search(r'Action:\s*(.*)', llm_output, re.DOTALL)
    if not action_match:
        observation = "Error: No action found in the agent's response."
        observation_str = f"Observation: {observation}"
        print(f"{observation_str}\n" + "="*40)
        prompt_history.append(observation_str)
        continue

    action_str = action_match.group(1).strip()

    if action_str.startswith('Finish['):
        final_answer = re.search(r'Finish\[(.*)\]', action_str, re.DOTALL).group(1).strip()
        print(f"Final Answer: {final_answer}\n")
        break

    tool_name = re.search(r"(\w+)\(", action_str).group(1)
    args_str = re.search(r"\((.*)\)", action_str).group(1)
    kwargs = dict(re.findall(r'(\w+)="([^"]*)"', args_str))

    if tool_name in available_tools:
        observation = available_tools[tool_name](**kwargs)
    else:
        observation = f":Eorrror: Tool '{tool_name}' is not available."

    # 3.4. 记录观察结果
    observation_str = f"Observation: {observation}"
    print(f"{observation_str}\n" + "="*40)
    prompt_history.append(observation_str)
    

Starting the agent...

User: I am planning to visit Paris. What's the weather like there and what attractions should I visit?

--- Step 1 ---
Agent Response:
Thought: I need to find the current weather in Paris in order to suggest suitable attractions. I will first get the weather information for Paris. 

Action: get_weather(city="Paris")

Observation: The current weather in Paris is Cloudy with a temperature of 8°C.
--- Step 2 ---
Agent Response:
Thought: Now that I have the current weather information for Paris, which is Cloudy with a temperature of 8°C, I can get a list of suitable tourist attractions based on these weather conditions.

Action: get_attractions(city="Paris", weather="Cloudy")

Observation: Based on the current weather, here are some popular tourist attractions in Paris:
- Weather in Paris, France: {'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.8667, 'lon': 2.3333, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1768858455, 'local